## Importing Libraries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## Calculating angles

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

## Knee Bend Counter

In [3]:
cap = cv2.VideoCapture('D:\Work & Study\Projects\Pose Estimation\KneeBend.mp4')

ptime = 0

bend_count = 0
stage = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        
        cv2.rectangle(frame, (0, 0), (210, 125), (0,0,0), -1, cv2.LINE_AA)
        
        
        ctime = time.time()
        fps = 1/(ctime-ptime)
        ptime = ctime
        cv2.putText(frame, 'FPS:' + str(int(fps)), (20, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            
            # Visualize angle
            cv2.putText(image, 'Left Leg Angle: ' + str(int(angle)), 
                           (20, 60), 
                           cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 2), 2)
            
            # bend counter 
            if angle > 160:
                stage = 'straight'
                cv2.putText(image, 'Keep your knee bent', (200, 460), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
            if angle <=120 and stage == 'straight':
                stage = 'bend'
                bend_count +=1
            
                    
            
        except:
            pass
        
        
        cv2.putText(image, 'Bend Count: ' + str(bend_count), (20, 100), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Knee Bend', image)

        if cv2.waitKey(10) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()